# **El patrón Repository**

In [28]:
class BaseModel:
    def __init__(self, id: int):
        self.id = id

    def __repr__(self):
        return f"{self.__class__.__name__}(id={self.id})"

In [29]:
from abc import ABC, abstractmethod
from typing import TypeVar, Generic, List

T = TypeVar('T', bound=BaseModel)

class Repository(ABC, Generic[T]):
    @abstractmethod
    def get(self, id: int) -> T:
        raise NotImplementedError

    @abstractmethod
    def get_all(self) -> List[T]:
        raise NotImplementedError

    @abstractmethod
    def add(self, **kwargs: object) -> None:
        raise NotImplementedError

    @abstractmethod
    def update(self, id: int, **kwargs: object) -> None:
        raise NotImplementedError

    @abstractmethod
    def delete(self, id: int) -> None:
        raise NotImplementedError


In [30]:
class InMemoryRepository(Repository[T]):
    def __init__(self, model_cls: type[T]):
        self.model_cls = model_cls
        self.entities = {}
        self.current_id = 1

    def get(self, id: int) -> T:
        return self.entities.get(id)

    def get_all(self) -> List[T]:
        return list(self.entities.values())

    def add(self, **kwargs: object) -> None:
        entity = self.model_cls(id=self.current_id, **kwargs)
        self.entities[self.current_id] = entity
        self.current_id += 1

    def update(self, id: int, **kwargs: object) -> None:
        if id in self.entities:
            for key, value in kwargs.items():
                setattr(self.entities[id], key, value)

    def delete(self, id: int) -> None:
        if id in self.entities:
            del self.entities[id]


In [31]:
class Product(BaseModel):
    def __init__(self, id: int, name: str, price: float):
        super().__init__(id)
        self.name = name
        self.price = price

    def __repr__(self):
        return f"Product(id={self.id}, name={self.name}, price={self.price})"

In [38]:
repo: Repository[Product]
repo.add(name='Product 1', price=10.0)
repo.add(name='Product 2', price=20.0)

 # Obtener un producto por ID
product = repo.get(1)
print(f"Get Product with ID 1: {product}")

Get Product with ID 1: None


In [33]:
def test_repository(repo: Repository[Product]):
    # Añadir productos
    repo.add(name='Product 1', price=10.0)
    repo.add(name='Product 2', price=20.0)

    # Obtener todos los productos
    products = repo.get_all()
    print(f"All Products: {products}")

    # Obtener un producto por ID
    product = repo.get(1)
    print(f"Get Product with ID 1: {product}")

    # Actualizar un producto
    repo.update(1, name='Updated Product 1', price=15.0)
    updated_product = repo.get(1)
    print(f"Updated Product with ID 1: {updated_product}")

    # Eliminar un producto
    repo.delete(1)
    deleted_product = repo.get(1)
    print(f"Deleted Product with ID 1: {deleted_product}")

In [35]:
product_repo = InMemoryRepository(Product)
test_repository(product_repo)

All Products: [Product(id=1, name=Product 1, price=10.0), Product(id=2, name=Product 2, price=20.0)]
Get Product with ID 1: Product(id=1, name=Product 1, price=10.0)
Updated Product with ID 1: Product(id=1, name=Updated Product 1, price=15.0)
Deleted Product with ID 1: None
